# Visualize File Database

FileDatabase: Consolidate JSON input videos + search function + Search in LLM output


## Initialization

In [ ]:
from pathlib import Path

from ai_xp.database import FileDatabase

inputs_lookup_dir_path = Path("../resources/inputs").resolve()
outputs_lookup_dir_path = Path("../generated/llm_output").resolve()

db = FileDatabase.from_paths(inputs_lookup_dir_path, outputs_lookup_dir_path)
db

## Demo

### Search

### Search Inputs

In [ ]:
db.search(db.inputs_dataframe, "CoMmeNt")

In [ ]:
db.search(db.inputs_dataframe, "monstre unicellulaire")


### Search Outputs

In [ ]:
db.search(db.outputs_dataframe, "CoMmeNt")

### Visualize All Inputs

In [ ]:
db.inputs_dataframe

### Visualize All Outputs

In [ ]:
db.outputs_dataframe

### Visualize Successful Outputs

In [ ]:
db.get_success_df()


### Visualize Errored Outputs

In [ ]:
db.get_errors_df()


### Visualize Inputs With Missing Outputs

Purely missing output files are missing outputs.

Errored output files also are missing outputs. It relies on the error suffix in output markdown file names : `{title_slug}.{exc_name}.err.md` instead of `{title_slug}.md` for success.

In [ ]:
db.inputs_with_missing_outputs().sort_values("exc_name")

Debug: Verify that latest markdown error files are used to provide error name.


In [ ]:
db.inputs_with_missing_outputs().loc[
    (
        db.inputs_with_missing_outputs()["exc_name"]
        != db.inputs_with_missing_outputs(keep="first")["exc_name"]
    )
]

Retry strategy: First download, no output file created yet (this is not a Retry but a Try strategy actually)

In [ ]:
db.inputs_with_missing_outputs().query("output_path.isna()")


Retry strategy: For NoTranscriptFound, try to query first available transcripts.

In [ ]:
db.inputs_with_missing_outputs().query(
    "exc_name in ('NoTranscriptFound')"
)


Query uncommon errors:

(likely `VideoUnavailable` meaning the URL is malformed, or the video was deleted.)

In [ ]:
db.inputs_with_missing_outputs().query(
    "(exc_name not in ('NoTranscriptFound', 'TranscriptsDisabled'))"
    "and output_path.notna()"
)
